In [4]:
import discum

In [6]:
servers = {
    "solaxy": {"guild": "948163452357214278", "channel": "948325458611687544"},
    "solspot": {"guild": "512333785338216465", "channel": "946899935352209438"}
}
authorizations = {
    "my": "NTI3MTM5MTg3NTQ0MjkzMzc3.YnEzvQ.EGbaAIdgKt4VA-NxxvGSrSkZzFY",
    "other1": ""
}
# Для получения id серверов и каналов нужно включить режим разработчика

Как работает бот?

<img style="display: float; float: left; padding-right: 2px; width: 400px; margin-right: 3px;" src="002.png">
Есть три компонента: бот, дискорд, юзер

1. юзер реагирует на сообщение, нажимая что-то в дисе (react)
2. дискорд принял ответ от пользователя, отправляет сообщение боту добавить ему роль (event)
3. бот назначает роль, посылает данные дискорду (add role)
4. юзер получает уведомление от дискорда о получении роли (notify)

Из `https://discord.com/developers/docs/topics/gateway` узнаем о следующей последовательности действий:
1. Открываем вебсокет
2. Получаем ответ (reply), состоящий из двух частей: identify ourselves и ping.
3. Получаем ответ ready от дискорда и мы можем прослушивать соединение.

Receiver 
1. get websocket message
2. call a handler

Sender
1. get a message from queue
2. send 

In [9]:
import websockets # удобнее, чем ванильный модуль socket, т.к. позволяет легко
# запускать запускать и слушать соединения в асинхронном режиме (там корутины, 
# вместо синхронных функций)
import asyncio
GATEWAY_URL = "wss://gateway.discord.gg/?v=9&encoding=json"

<img style="display: float; float: left; padding-right: 3px; width: 420px;" src="001.png"> Смотрим раздел Payloads (полезная нагрузка) - она отправляется при установлении соединения. Т.к. вся эта "полезная нагрузка" отправляется в формате JSON по вебсокету, удобно поместить список параметров в `@dataclass`

In [4]:
from dataclasses import dataclass

@dataclass
class GatewayMessage():
    op: int
    data: object
    sequence: int
    name: str

Написать просто `async with websockets.connect(GATEWAY_URL) as ws:` мы не можем, это должно быть внутри асинхронной функции.

In [11]:
async def run_connection():
    async with websockets.connect(GATEWAY_URL) as ws:
        async for msg in ws:
            print(msg)

task1 = asyncio.create_task(run_connection())

{"t":null,"s":null,"op":10,"d":{"heartbeat_interval":41250,"_trace":["[\"gateway-prd-main-2ssv\",{\"micros\":0.0}]"]}}


<img src="003.png">
Можно еще так написать, хз в чем отличие 

Пишем три важные составляющие любого соединения: receiver, sender, ping

In [13]:
bot = discum.Client(token="NTI3MTM5MTg3NTQ0MjkzMzc3.YnEzvQ.EGbaAIdgKt4VA-NxxvGSrSkZzFY")

Retrieving Discord's build number...
Could not retrieve discord build number.
invalid literal for int() with base 10: 'ull!=='
 [+] (<discum.start.login.Login->getXFingerprint) Get -> https://discord.com/api/v9/experiments
 [+] (<discum.start.login.Login->getXFingerprint) Response <- b'{"fingerprint": "971363433494745088.JrwDrbEbXZAFRv92QYDTYaPZXCs", "assignments": [[70411085, 0, 1, -1, 0], [16416918, 0, 1, -1, 0], [3816091942, 3, 0, -1, 0], [4130837190, 0, 10, -1, 0], [852550504, 3, 1, -1, 0], [2599708267, 0, 1, -1, 0], [1365487849, 6, 1, -1, 0], [3128009767, 0, 10, -1, 0], [2973729510, 2, 1, -1, 0], [2571931329, 1, 6, -1, 0], [1260103069, 0, 1, -1, 0], [2003494159, 1, 1, -1, 0], [2458926251, 1, 6, -1, 2], [4027800, 1, 6, -1, 3], [51193042, 0, 1, -1, 0], [2063302207, 1, 1, -1, 0], [2634540382, 3, 1, -1, 0], [427713714, 0, 1, -1, 0], [3316720711, 0, 1, -1, 1], [156590431, 0, 1, -1, 2], [1126477623, 0, 1, -1, 0], [2617218444, 4, 1, -1, 2], [3615263239, 0, 1, -1, 0], [414823706, 0, 1, -1

In [10]:
bot = discum.Client(token=authorizations["my"])

Retrieving Discord's build number...
Could not retrieve discord build number.
invalid literal for int() with base 10: 'ull!=='
 [+] (<discum.start.login.Login->getXFingerprint) Get -> https://discord.com/api/v9/experiments
 [+] (<discum.start.login.Login->getXFingerprint) Response <- b'{"fingerprint": "971332432223686696._MwAPdUb56o51ifpnwOM6zSvsS0", "assignments": [[70411085, 0, 1, -1, 0], [16416918, 0, 1, -1, 0], [3816091942, 3, 2, -1, 0], [4130837190, 0, 10, -1, 0], [852550504, 3, 1, -1, 0], [2599708267, 0, 1, -1, 0], [1365487849, 6, 1, -1, 0], [3128009767, 0, 10, -1, 0], [2973729510, 2, 1, -1, 0], [2571931329, 1, 6, -1, 0], [1260103069, 0, 1, -1, 0], [2003494159, 1, 1, -1, 0], [2458926251, 1, 6, -1, 2], [4027800, 1, 6, -1, 3], [51193042, 0, 1, -1, 0], [2063302207, 1, 1, -1, 0], [2634540382, 3, 1, -1, 0], [427713714, 0, 1, -1, 0], [3316720711, 0, 1, -1, 1], [156590431, 0, 1, -1, 2], [1126477623, 0, 1, -1, 0], [2617218444, 4, 1, -1, 2], [3615263239, 0, 1, -1, 0], [414823706, 0, 1, -1

In [8]:
s = 'Валя где ты шляешься, это сообщение отправлено через жупутур, быстра вовзращайся падла'
for i in [s[i]+s[i+1]+s[i+2] for i in range(0, len(s)-2, 3)]:
    
    bot.sendMessage('616695996436447251', i)

 [+] (<discum.messages.messages.Messages->sendMessage) Post -> https://discord.com/api/v9/channels/616695996436447251/messages
 [+] (<discum.messages.messages.Messages->sendMessage) {"content": "\u0412\u0430\u043b", "tts": false, "nonce": "971330639888384000"}
 [+] (<discum.messages.messages.Messages->sendMessage) Response <- b'{"message": "Unknown Channel", "code": 10003}'
 [+] (<discum.messages.messages.Messages->sendMessage) Post -> https://discord.com/api/v9/channels/616695996436447251/messages
 [+] (<discum.messages.messages.Messages->sendMessage) {"content": "\u044f \u0433", "tts": false, "nonce": "971330644082688000"}
 [+] (<discum.messages.messages.Messages->sendMessage) Response <- b'{"message": "Unknown Channel", "code": 10003}'
 [+] (<discum.messages.messages.Messages->sendMessage) Post -> https://discord.com/api/v9/channels/616695996436447251/messages
 [+] (<discum.messages.messages.Messages->sendMessage) {"content": "\u0434\u0435 ", "tts": false, "nonce": "9713306440826880

In [9]:
bot.sendMessage('616695996436447251', 'Hello')

 [+] (<discum.messages.messages.Messages->sendMessage) Post -> https://discord.com/api/v9/channels/616695996436447251/messages
 [+] (<discum.messages.messages.Messages->sendMessage) {"content": "Hello", "tts": false, "nonce": "971331575218176000"}
 [+] (<discum.messages.messages.Messages->sendMessage) Response <- b'{"message": "Unknown Channel", "code": 10003}'


<Response [404]>

In [ ]:
def close_after_fetching(resp, guild_id):
    if bot.gateway.finishedMemberFetching(guild_id):
        lenmembersfetched = len(bot.gateway.session.guild(guild_id).members)  # this line is optional
        print(str(lenmembersfetched)+' members fetched')
        bot.gateway.removeCommand({'function': close_after_fetching, 'params': {'guild_id': guild_id}})
        bot.gateway.close()

In [ ]:
def get_members(guild_id, channel_id):
    bot.gateway.fetchMembers(guild_id, channel_id, keep="all", wait=1)  # get all user attributes, wait 1 second between requests
    bot.gateway.command({'function': close_after_fetching, 'params': {'guild_id': guild_id}})
    bot.gateway.run()
    bot.gateway.resetSession()  # saves 10 seconds when gateway is run again
    return bot.gateway.session.guild(guild_id).members

In [ ]:
members = get_members(guild_id='948163452357214278', channel_id='948325458611687544')
memberslist = []

for memberID in members:
    memberslist.append(memberID)
    print(memberID)

f = open('users.txt', "a")
for element in memberslist:
    f.write(element + '\n')
f.close()

In [ ]:
bot.getOauth2Tokens()